In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

In [2]:
dataset = '../model/sign_classifier/sign.csv'
model_save_path = '../model/sign_classifier/keypoint_classifier.h5'

In [6]:
NUM_CLASSES = 2
DATA_LEN = 21 * 3

In [18]:
data = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, DATA_LEN + 1)))
data.shape

(308, 63)

In [19]:
lables = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))
lables.shape

(308,)

In [20]:
data_train, data_test, label_train, label_test = train_test_split(
    data, lables, 
    train_size=0.75, 
    random_state=RANDOM_SEED
)
# print(data_train.shape, data_test.shape, label_train.shape, label_test.shape)

## Model

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((DATA_LEN, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 63)                0         
                                                                 
 dense (Dense)               (None, 20)                1280      
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 2)                 22        
                                                                 
Total params: 1,512
Trainable params: 1,512
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [12]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

## Model training

In [21]:
model.fit(
    data_train,
    label_train,
    epochs=1000,
    batch_size=128,
    validation_data=(data_test, label_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
1/2 [==============>...............] - ETA: 0s - loss: 0.6900 - accuracy: 0.5703
Epoch 1: saving model to model/sign_classifier\keypoint_classifier.hdf5
2/2 [==============================] - 0s 260ms/step - loss: 0.6906 - accuracy: 0.5541 - val_loss: 0.6843 - val_accuracy: 0.5974
Epoch 2/1000
1/2 [==============>...............] - ETA: 0s - loss: 0.6880 - accuracy: 0.5391
Epoch 2: saving model to model/sign_classifier\keypoint_classifier.hdf5
2/2 [==============================] - 0s 60ms/step - loss: 0.6902 - accuracy: 0.5455 - val_loss: 0.6822 - val_accuracy: 0.5844
Epoch 3/1000
1/2 [==============>...............] - ETA: 0s - loss: 0.6915 - accuracy: 0.5469
Epoch 3: saving model to model/sign_classifier\keypoint_classifier.hdf5
2/2 [==============================] - 0s 54ms/step - loss: 0.6911 - accuracy: 0.5455 - val_loss: 0.6801 - val_accuracy: 0.5714
Epoch 4/1000
1/2 [==============>...............] - ETA: 0s - loss: 0.6850 - accuracy: 0.6406
Epoch 4: saving model t

In [23]:
# Model evaluation
val_loss, val_acc = model.evaluate(data_test, label_test, batch_size=128)

1/1 [==============================] - 0s 29ms/step - loss: 0.0541 - accuracy: 0.9870


In [26]:
# Inference test
predict_result = model.predict(np.array([data_test[2]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 32ms/step
[0.99345183 0.00654817]
0


## Save model

In [29]:
model.save(model_save_path)